# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [133]:
import pandas as pd
import plotly.express as px

# Notebook Presentation

In [2]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [52]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [ ]:
# --- Get the number of rows and columns in the dataframe:
df_apps.shape # 10841 rows, 12 columns.

In [ ]:
# --- Get all of the column names:
df_apps.columns

In [ ]:
# --- Get five random rows from the dataframe:
df_apps.sample(n=5)

### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [55]:
# --- Remove "Last_Updated" and "Android_Ver" (not called Android_Version) from the dataframe:
df_apps.drop(columns=["Last_Updated", "Android_Ver"], axis=1, inplace=True)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [ ]:
# --- Count the number of rows that are missing a review score:
df_apps["Rating"].isna().sum()

In [ ]:
# --- Remove the row(s) that contain a NaN value for the Rating:
df_apps_clean = df_apps.dropna(axis=0, subset=["Rating"])

# --- Count the number of rows that are missing a review score (should be 0):
print(df_apps_clean["Rating"].isna().sum())

# --- Check the number of rows and columns (should be 9367 and 10)
print(df_apps_clean.shape)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [ ]:
# --- Show how many dupicates there are:
print(f"There are { df_apps_clean.duplicated().sum() } duplicates")

In [ ]:
# --- Show the duplicates of the App named
df_apps_clean[df_apps_clean["App"] == "Instagram"]

In [111]:
# --- Find and remove the duplicates in the App column and return as a new variable:
df_apps_clean = df_apps_clean.drop_duplicates(subset=["App"], keep="first")

In [ ]:
# --- Show the number of rows and columns:
df_apps_clean.shape

In [ ]:
# --- Display the rows containing Instagram in the App column:
df_apps_clean[df_apps_clean["App"] == "Instagram"]

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [ ]:
# --- Display the top 10 apps by the highest ratings:
df_apps_clean.sort_values(by=["Rating"], ascending=False).head(n=10)

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [ ]:
# --- Display the top 50 apps by the size of the app in MB's:
df_apps_clean.sort_values(by=["Size_MBs"], ascending=False).head(n=5)
# --- Result: There could be a 100MB limit.

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [ ]:
# --- Display the top 50 apps by the number of reviews:
df_apps_clean.sort_values(by=["Reviews"], ascending=False).head(n=50)

# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [131]:
    # --- Count the number of app ratings and display the value next to each rating:
    ratings = df_apps_clean.Content_Rating.value_counts()
    print(ratings)

Everyone           6619
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: Content_Rating, dtype: int64


In [145]:
# --- Present the ratings data as a pie chart:

fig = px.pie(labels = ratings.index, 
             values = ratings.values,
             title = "Content Rating",
             names = ratings.index
             )

fig.update_traces(textposition = "outside", 
                  textinfo = "percent+label")

fig.show()

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



In [148]:
# --- Show the same data again but as a doughnut chart.
fig = px.pie(labels = ratings.index, 
             values = ratings.values,
             title = "Content Rating",
             names = ratings.index,
             # Make the chart a doughnut using hole=:
             hole=0.5
             )

fig.update_traces(textposition = "inside", 
                  textinfo = "percent",
                  textfont_size = 16)

fig.show()

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [155]:
# --- Show the datatypes of each column:
df_apps_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8197 entries, 21 to 10835
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8197 non-null   object 
 1   Category        8197 non-null   object 
 2   Rating          8197 non-null   float64
 3   Reviews         8197 non-null   int64  
 4   Size_MBs        8197 non-null   float64
 5   Installs        8197 non-null   object 
 6   Type            8197 non-null   object 
 7   Price           8197 non-null   object 
 8   Content_Rating  8197 non-null   object 
 9   Genres          8197 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 962.5+ KB


In [159]:
# --- Find and remove any , from the installs column:
df_apps_clean["Installs"] = df_apps_clean.Installs.astype(str).str.replace(",", "")

In [160]:
# --- Convert the type of the Installs column from string to numeric:
df_apps_clean["Installs"] = pd.to_numeric(df_apps_clean.Installs)

In [161]:
# Show the complete list of app installs by count:
df_apps_clean[["App", "Installs"]].groupby("Installs").count()

,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,698
5000,425
10000,988


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [169]:
# --- They all have a $ in front so they need to be removed:
df_apps_clean["Price"] = df_apps_clean.Installs.astype(str).str.replace("$", "")
df_apps_clean["Price"] = pd.to_numeric(df_apps_clean.Installs)

# --- Display the list of prices:
df_apps_clean[["App", "Price"]].groupby("Price").count()


,App
Price,
1,3
5,9
10,69
50,56
100,303
500,199
1000,698
5000,425
10000,988


### The most expensive apps sub $250

In [170]:
df_apps_clean.sort_values(by="Price", ascending = False).head(20)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10835,Subway Surfers,GAME,4.50,27722264,76.00,1000000000,Free,1000000000,Everyone 10+,Arcade
10806,Instagram,SOCIAL,4.50,66577313,5.30,1000000000,Free,1000000000,Teen,Social
10783,Google Play Books,BOOKS_AND_REFERENCE,3.90,1433233,5.70,1000000000,Free,1000000000,Teen,Books & Reference
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,1000000000,Free,1000000000,Everyone,Communication
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,1000000000,Free,1000000000,Everyone,Communication
10786,Google Chrome: Fast & Secure,COMMUNICATION,4.30,9642995,3.50,1000000000,Free,1000000000,Everyone,Communication
10788,Hangouts,COMMUNICATION,4.00,3419249,3.50,1000000000,Free,1000000000,Everyone,Communication
10792,Skype - free IM & video calls,COMMUNICATION,4.10,10484169,3.50,1000000000,Free,1000000000,Everyone,Communication
10803,Google Play Games,ENTERTAINMENT,4.30,7165362,9.35,1000000000,Free,1000000000,Teen,Entertainment
10805,Facebook,SOCIAL,4.10,78158306,5.30,1000000000,Free,1000000000,Teen,Social


### Highest Grossing Paid Apps (ballpark estimate)

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

### Vertical Bar Chart - Highest Competition (Number of Apps)

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log') 

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). 


# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

# Grouped Bar Charts: Free vs. Paid Apps per Category

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart: 

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart. 

<img src=https://imgur.com/uVsECT3.png>


# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below: 

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.